In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import botorch
from pak.choose import *
from pak.function import *
from pak.GPmodel import fit_gp_model
from pak.plotfunc import visualize_2d_contour
from pak.AugmentedEI import AugmentedExpectedImprovement
from skopt.sampler import Lhs
import warnings
warnings.filterwarnings("ignore")
torch.set_default_tensor_type(torch.DoubleTensor)

func = 'Fn7'
noise = 'M'
funcname = func + noise

#### 定义函数，搜索空间和预算个数

In [2]:
fn = choose(funcname)[0]
f = choose(funcname)[1]
search_space = [[-5.0, 5.0], [-5.0, 5.0]] 
bounds = torch.tensor([[-5.0, -5.0], [5.0, 5.0]])
budgets = 200

#### 初始化(拉丁超立方采样)

In [3]:
lhs = Lhs()
samples = lhs.generate(search_space, n_samples=round(0.1*budgets))
train_x = torch.tensor(samples)
train_y = []
for x in train_x:
    train_y.append(fn(x.numpy()))
train_y = torch.tensor(train_y)

obs_x = torch.tensor(samples)

In [4]:
for i in tqdm(range(budgets-len(samples))):

    model, likelihood = fit_gp_model(train_x, train_y)

    policy = AugmentedExpectedImprovement(
        model, best_f=model(train_x).mean.min(), maximize = False
    )

    next_x, acq_val = botorch.optim.optimize_acqf( 
        policy,
        bounds=bounds,
        q=1,
        num_restarts=40,
        raw_samples=100,
    )

    next_x_obs = train_x[train_y.argmin()].reshape(1, 2)

    next_y = torch.tensor(fn(next_x.numpy()))

    train_x = torch.cat([train_x, next_x])
    train_y = torch.cat([train_y, next_y])

    obs_x  = torch.cat([obs_x, next_x_obs])   

  0%|          | 0/180 [00:00<?, ?it/s]

TypeError: __init__() got an unexpected keyword argument 'posterior_transform'

In [5]:
x1_range = torch.linspace(-5, 5, 1000)
x2_range = torch.linspace(-5, 5, 1000)
X1, X2 = torch.meshgrid(x1_range, x2_range, indexing="ij")
xs = torch.vstack((X1.flatten(), X2.flatten())).transpose(-1, -2)
suggest_x = train_x[train_y.argmin()].numpy()
suggest_y = f(suggest_x)
suggest = [suggest_x,suggest_y]

In [ ]:
data = f(xs.numpy())
visualize_2d_contour(name='Objective function '+funcname+'(noisefree)',x1_range=x1_range,x2_range=x2_range, data=data, train_x=train_x,labelx="AEI")

In [ ]:
data = model(xs).mean.detach().numpy()
visualize_2d_contour(name='Predictive function '+funcname,x1_range=x1_range,x2_range=x2_range,data=data,train_x=train_x,suggest=suggest,labelx="EI")

In [ ]:
data = model(xs).mean.detach().numpy()
visualize_2d_contour(name='Predictive function '+funcname,x1_range=x1_range,x2_range=x2_range,data=data,train_x=obs_x[round(0.1*budgets):],suggest=suggest,labelx="obs")